# Import libraries

In [1]:
import numpy as np
import random
import math
import seaborn as sns
from src.smithMethods import *
from src.utils.delaunay2d import *
from src.utils.hyperbolicWrappedGaussian import disc_pt_to_hyperboloid, hyperbolic_sampling, proj
from scipy.spatial.qhull import QhullError
from collections import defaultdict 
from phcpy.phcpy2c3 import py2c_set_seed
import time
from matplotlib.lines import Line2D
import networkx as nx
import json
from pathlib import Path
import pickle 
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra
import statistics
import biotite.sequence.phylo as phylo
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

FIG_PATH =  Path("./Figures/Results")

PHCv2.4.86 released 2022-05-30 works!


# Functions

In [3]:
def plotSteinerTree(steinerGraph, verticesDict, mstGraph, edgesDT, space="Klein", additional="MST", title='Title'):
    
    fig = plt.figure(figsize=(6,6), dpi=200)
    ax = fig.add_subplot(111, aspect='equal') 
    
    if space=="Klein" or space=="Poincare":
        circ = plt.Circle((0, 0), radius=1, edgecolor='black', facecolor='None', linewidth=3, alpha=0.5)
        ax.add_patch(circ)
        
    if mstGraph is not None or edgesDT is not None:
        if additional == "MST":
            H = nx.Graph(mstGraph)
        elif additional == "DT":
            H = nx.Graph([[f"T{p}" for p in edge] for edge in edgesDT])
        else:
            raise ValueError("additional should either 'MST' or 'DT'")

        nx.draw_networkx_edges(H, pos=verticesDict, style='--', alpha=0.4)
    
    G = nx.Graph(steinerGraph)
    G.add_node("T0")
    color_map = []

    for node in G:
        if node[0]=="S":
            color_map.append('tab:blue')
        else: 
            color_map.append('tab:red') 

    nx.draw(G, node_color=color_map, pos=verticesDict,node_size=5) #node_size=50
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Terminal',markerfacecolor='tab:red', markersize=5), #markersize=12
        Line2D([0], [0], marker='o', color='w', label='Steiner',markerfacecolor='tab:blue', markersize=5), #markersize=12       
    ]

    
    plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.10, 1))
    plt.tight_layout()
    
    #fig.savefig(FIG_PATH / f"{space}_dtSMT.png")
    
    plt.savefig(title+'.pdf')
    #plt.show()

def klein2poincare(z_klein):
    norm_squared = z_klein[0]**2+z_klein[1]**2 
    denom = 1+np.sqrt(1-norm_squared)
    z_poincare = z_klein[0]/denom, z_klein[1]/denom
    return np.array(z_poincare)

def verticesDict_klein2poincare(verticesDict):
    verticesDict_Poincare = verticesDict
    for key in verticesDict_Poincare.keys():
        verticesDict_Poincare[key] = klein2poincare(verticesDict_Poincare[key])
    return verticesDict_Poincare

def distance_klein(p,q): #with p and q in euclidean coordinates and their norm<1
    norm_p = (p**2).sum(-1)
    norm_q = (q**2).sum(-1)
    dot_pq = (p*q).sum(-1)
    return np.arccosh((1-dot_pq)/np.sqrt((1-norm_p)*(1-norm_q)))  

def distance_matrix(pts, dist=distance_klein):
    return dist(np.expand_dims(pts, 0), np.expand_dims(pts, 1)) 

def build_hyperbolically_weighted_adjacency_matrix(Graph, verticesDict, total_nmbr_terminal = 2047):
    #it can be that some terminals are missing in verticesDict. 
    max_idx_steiner = 0
    for edge in Graph:
        for vertex in edge:
            if list(vertex)[0] == 'S':
                idx_steiner = ''
                for l in vertex[1:]:
                    idx_steiner=idx_steiner+l 
                idx_steiner = int(idx_steiner)                
                if idx_steiner > max_idx_steiner:
                    max_idx_steiner = idx_steiner
    total_nmbr_steiner = max_idx_steiner + 1 
    nmbr_nodes = total_nmbr_terminal + total_nmbr_steiner
    hyperbolically_weighted_adjacency_matrix = np.zeros((nmbr_nodes, nmbr_nodes))    
    
    for edge in Graph:
        indices = []
        
        for vertex in edge:
            
            nmbr_str = ''
            for l in vertex[1:]:
                nmbr_str=nmbr_str+l 
            nmbr_integer = int(nmbr_str)
            
            if list(vertex)[0] == 'T':
                index = nmbr_integer
            else: #i.e. list(vertex)[0] == 'S'
                index = total_nmbr_terminal + nmbr_integer
            indices.append(index)
        
        hyperbolic_dist_edge = distance_klein(verticesDict[edge[0]],verticesDict[edge[1]])
        hyperbolically_weighted_adjacency_matrix[indices[0]][indices[1]] = hyperbolic_dist_edge 
        hyperbolically_weighted_adjacency_matrix[indices[1]][indices[0]] = hyperbolic_dist_edge 

    return hyperbolically_weighted_adjacency_matrix

def normalize(matrix): #or vector
    normalized_matrix = (1/np.max(matrix))*matrix
    return normalized_matrix

def normalize_embedding(z):

    #need to normalize to get norms<1 in order to build the Hyperbolic Voronoi Diagram
    z_norms=[]
    for v in z:
        z_norms.append(np.linalg.norm(v))
    max_z_norms = max(z_norms)+0.001 #small perturbation of 0.001 to not have the maximal norm being normalized to exactly 1 because all the normalized norms should be stricly inferior to 1   

    z_normalized=[]
    for i in range(len(z)):
            z_normalized.append([z[i][0]/max_z_norms, z[i][1]/max_z_norms])

    return z_normalized

def poincare2klein(z_normalized):
    #Convert from Poincaré to Beltrami-Cayley-Klein model latent representation
    z_beltrami = []
    for i in range(len(z_normalized)):
        norm_squared = z_normalized[i][0]**2+z_normalized[i][1]**2 
        z_beltrami.append([2*z_normalized[i][0]/(1+norm_squared), 2*z_normalized[i][1]/(1+norm_squared)])

    z_beltrami=np.array(z_beltrami)
    return z_beltrami

def get_disconnected_indices(planaria_klein, verticesDict):
    disconnected_indices = [] 
    for i in range(planaria_klein.shape[0]):
        if 'T'+str(i) not in verticesDict.keys():
            disconnected_indices.append(i) 
    return disconnected_indices

def build_nj_adjacency_matrix(nj_graph, total_nmbr_terminal = 2047):

    max_idx_steiner = 0
    for edge in nj_graph:
        for vertex in edge:
            if list(vertex)[0] == 'S':
                idx_steiner = ''
                for l in vertex[1:]:
                    idx_steiner=idx_steiner+l 
                idx_steiner = int(idx_steiner)                
                if idx_steiner > max_idx_steiner:
                    max_idx_steiner = idx_steiner
    total_nmbr_steiner = max_idx_steiner + 1 
    nmbr_nodes = total_nmbr_terminal + total_nmbr_steiner
    nj_adjacency_matrix = np.zeros((nmbr_nodes, nmbr_nodes))
    
    for edge in nj_graph:
        indices = []
        
        for vertex in edge:
            
            nmbr_str = ''
            for l in vertex[1:]:
                nmbr_str=nmbr_str+l 
            nmbr_integer = int(nmbr_str)
            
            if list(vertex)[0] == 'T':
                index = nmbr_integer
            else: #i.e. list(vertex)[0] == 'S'
                index = total_nmbr_terminal + nmbr_integer
            indices.append(index)
        
        dist_edge = nj_graph[edge]
        nj_adjacency_matrix[indices[0]][indices[1]] = dist_edge 
        nj_adjacency_matrix[indices[1]][indices[0]] = dist_edge 

    return nj_adjacency_matrix

def neighbor_joining(points, dist_matrix, space="Klein"):
    if space not in ["Klein", "Euclidean", "Half"]:
        raise ValueError("space should be either 'Klein', 'Euclidean' or 'Half'")
    
    tree = phylo.neighbor_joining(dist_matrix)
    G = tree.as_graph().to_undirected()
    
    mapping = dict()
    count = 0
    for n in G.nodes:
        if type(n) is tuple:
            mapping[n] = f"S{count}"
            count+=1
        else:
            mapping[n] = f"T{n}"
    
    G = nx.relabel_nodes(G, mapping)

    return nx.get_edge_attributes(G,'distance')


def run_realbio_exp(percentage, planaria_klein, planaria_pseudotime_groundtruth):
# This is the percentage of missing data i.e. (100-percentage)% of data are considered
    threshold = int((100-percentage)/100 * planaria_klein.shape[0])

    nmbr_permutation = 100 # number of permutations i.e. of different random sampling of the data


    random_seed = 0
    random.seed(random_seed)
    np.random.seed(random_seed)

    root_idx = 0


    ######################

    corr_dict = {'steiner':[], 'mst': [], 'nj':[]}
    frobenius_dict = {'steiner':[], 'mst': [], 'nj':[]}
    time_dict = {'steiner':[], 'mst': [], 'nj':[]}

    print('random_seed', random_seed)
    print('percentage', percentage)
    print()

    for permutation in range(nmbr_permutation):
        print('permutation', permutation)

        # We keep the root index 0 at 0, and permute the rest
        permutation_indices = np.random.permutation(np.arange(planaria_klein.shape[0])[1:])
        permutation_indices = np.append(root_idx, permutation_indices)

        points_klein = planaria_klein[permutation_indices, :][:threshold]
        pseudotime_groundtruth = planaria_pseudotime_groundtruth[permutation_indices][:threshold]
        #normalize pseudotime_groundtruth
        pseudotime_groundtruth_normalized = normalize(pseudotime_groundtruth)
        nmbr_terminal=points_klein.shape[0]

        #NJ
        start_time = time.time()
        #print('Computing distance matrix for NJ...')
        distance_matrix_points_klein = distance_matrix(points_klein)
        njGraph = neighbor_joining(points_klein, distance_matrix_points_klein) 
        adjacency_matrix_nj = build_nj_adjacency_matrix(njGraph, total_nmbr_terminal=nmbr_terminal)

        csr_adjacency_matrix_nj = csr_matrix(adjacency_matrix_nj)
        del adjacency_matrix_nj
        #only distance to the root which is at index root_idx
        shortest_path_distance_matrix_nj_all = dijkstra(csgraph=csr_adjacency_matrix_nj, directed=False, indices=root_idx)
        shortest_path_distance_matrix_nj_terminal = shortest_path_distance_matrix_nj_all[:nmbr_terminal]
        normalized_shortest_path_distance_matrix_nj_terminal = normalize(shortest_path_distance_matrix_nj_terminal)
        end_time = time.time()
        time_nj = end_time - start_time
        corr_nj = np.corrcoef(pseudotime_groundtruth_normalized, normalized_shortest_path_distance_matrix_nj_terminal)[0][1] 
        frobenius_nj = np.linalg.norm(normalized_shortest_path_distance_matrix_nj_terminal-pseudotime_groundtruth_normalized)
        #print('corr_nj', corr_nj)
        print('frobenius_nj', frobenius_nj)
        print('time_nj', time_nj)


        start_time = time.time()
        steinerGraph, verticesDict, ratio, mstGraph, mstTime, edgesDT, FST3, numFST4 = heuristicMSTpipeline(points_klein, 
                                                                                   space="Klein", 
                                                                                   triangMeth="DT", 
                                                                                   maxgroup=4, 
                                                                                   nIters=3, 
                                                                                   convDiff=1e-1, 
                                                                                   dist2Points=1e-5,
                                                                                   precise=True)
        #plotSteinerTree(steinerGraph, verticesDict, mstGraph, edgesDT, space="Klein", title='Planaria_rdmseed'+str(random_seed)+'_percent'+str(percentage)+'_permutation'+str(permutation)+'_Klein') 
        #verticesDict_Poincare = verticesDict_klein2poincare(verticesDict)
        #plotSteinerTree(steinerGraph, verticesDict_Poincare, mstGraph, edgesDT, space="Poincare", title='Planaria_rdmseed'+str(random_seed)+'_percent'+str(percentage)+'_permutation'+str(permutation)+'_Poincare') 

        hyperbolically_weighted_adjacency_matrix_steiner = build_hyperbolically_weighted_adjacency_matrix(steinerGraph, verticesDict, total_nmbr_terminal=nmbr_terminal)

        csr_hyperbolically_weighted_adjacency_matrix_steiner = csr_matrix(hyperbolically_weighted_adjacency_matrix_steiner)
        del hyperbolically_weighted_adjacency_matrix_steiner
        #only distance to the root which is at index 6
        shortest_path_distance_matrix_steiner_all = dijkstra(csgraph=csr_hyperbolically_weighted_adjacency_matrix_steiner, directed=False, indices=root_idx)
        shortest_path_distance_matrix_steiner_terminal = shortest_path_distance_matrix_steiner_all[:nmbr_terminal]
        normalized_shortest_path_distance_matrix_steiner_terminal = normalize(shortest_path_distance_matrix_steiner_terminal)    
        end_time = time.time()
        time_steiner = end_time - start_time

        start_time = time.time()
        hyperbolically_weighted_adjacency_matrix_mst = build_hyperbolically_weighted_adjacency_matrix(mstGraph, verticesDict, total_nmbr_terminal=nmbr_terminal)         
        csr_hyperbolically_weighted_adjacency_matrix_mst = csr_matrix(hyperbolically_weighted_adjacency_matrix_mst)
        del hyperbolically_weighted_adjacency_matrix_mst
        shortest_path_distance_matrix_mst_all = dijkstra(csgraph=csr_hyperbolically_weighted_adjacency_matrix_mst, directed=False, indices=root_idx)
        shortest_path_distance_matrix_mst_terminal = shortest_path_distance_matrix_mst_all[:nmbr_terminal]
        normalized_shortest_path_distance_matrix_mst_terminal = normalize(shortest_path_distance_matrix_mst_terminal)
        end_time = time.time()
        time_mst = end_time - start_time + mstTime
        
        corr_steiner = np.corrcoef(pseudotime_groundtruth_normalized, normalized_shortest_path_distance_matrix_steiner_terminal)[0][1] 
        frobenius_steiner = np.linalg.norm(normalized_shortest_path_distance_matrix_steiner_terminal-pseudotime_groundtruth_normalized)

        corr_mst = np.corrcoef(pseudotime_groundtruth_normalized, normalized_shortest_path_distance_matrix_mst_terminal)[0][1] 
        frobenius_mst = np.linalg.norm(normalized_shortest_path_distance_matrix_mst_terminal-pseudotime_groundtruth_normalized)

        #print('corr_steiner', corr_steiner)
        print('frobenius_steiner', frobenius_steiner)
        print('time_steiner', time_steiner)

        #print('corr_mst', corr_mst)   
        print('frobenius_mst', frobenius_mst)
        print('time_mst', time_mst)
        print()

        corr = {'nj':corr_nj, 'steiner':corr_steiner, 'mst':corr_mst}
        frobenius = {'nj':frobenius_nj, 'steiner':frobenius_steiner, 'mst':frobenius_mst}
        cpu_time = {'nj':time_nj, 'steiner':time_steiner, 'mst':time_mst}

        for key in ['nj', 'steiner', 'mst']:
            corr_dict[key].append(corr[key])
            frobenius_dict[key].append(frobenius[key])
            time_dict[key].append(cpu_time[key])

    # Save results
    name = 'Results/Real/Planaria_rdmseed'+str(random_seed)+'_percent'+str(percentage)+'_nmbrpermutation'+str(nmbr_permutation) 
    with open(name+'_corr_dict.pkl', 'wb') as f:
        pickle.dump(corr_dict, f)  
    with open(name+'_frobenius_dict.pkl', 'wb') as f:
        pickle.dump(frobenius_dict, f)
    with open(name+'_time_dict.pkl', 'wb') as f:
        pickle.dump(time_dict, f)

    print('DONE!') 
    non_valid_perm = [] # this is to remove samplings leading to numerical issues for the MST and Steiner
    for i in range(len(corr_dict['steiner'])):
        if math.isnan(corr_dict['steiner'][i]):
            non_valid_perm.append(i) 
    valid_perm = np.arange(len(corr_dict['steiner']))
    valid_perm = np.delete(valid_perm, non_valid_perm)
    print('non_valid_perm:', non_valid_perm)    

    print('random seed:', random_seed)
    print('percentage:', percentage)
    print('number of permutations:', nmbr_permutation)
    max_perm = nmbr_permutation # option to consider the max_perm first permutations
    print('max perm', max_perm)
    print()
    for key in ['nj', 'steiner', 'mst']:
        print(key)
        #print('corr mean+-std '+str(statistics.mean(np.array(corr_dict[key])[valid_perm][:max_perm]))+'+-'+str(statistics.stdev(np.array(corr_dict[key])[valid_perm][:max_perm])))
        print('frobenius mean+-std '+str(statistics.mean(np.array(frobenius_dict[key])[valid_perm][:max_perm]))+'+-'+str(statistics.stdev(np.array(frobenius_dict[key])[valid_perm][:max_perm])))    
        print('time mean+-std '+str(statistics.mean(np.array(time_dict[key])[valid_perm][:max_perm]))+'+-'+str(statistics.stdev(np.array(time_dict[key])[valid_perm][:max_perm])))    
        print()

def run_realbio_exp_all(planaria_klein, planaria_pseudotime_groundtruth): #to run from percentages 95 to 75, by step of 2.5, and for 100 samplings at each percentage
    for i in range(95, 74, -5):
        run_realbio_exp(i, planaria_klein, planaria_pseudotime_groundtruth)
        if i > 75:
            run_realbio_exp(i-2.5, planaria_klein, planaria_pseudotime_groundtruth)   

# Load processed data and run experiments

In [ ]:
SAVE_PATH = Path("./Data")

In [4]:
with open(SAVE_PATH / 'planaria_klein_cleaned.npy', 'rb') as f:
    data = np.load(f)

In [5]:
with open(SAVE_PATH / 'planaria_pseudotime_groundtruth_cleaned.npy', 'rb') as f:
    groundtruth = np.load(f)

In [7]:
run_realbio_exp_all(data, groundtruth)

# Load the results and rearrange the data structure

In [16]:
# Load results
random_seed = 0
nmbr_permutation = 100 # number of permutations
percentages = [95, 92.5, 90, 87.5, 85, 82.5, 80, 77.5, 75]
results = {}
for percentage in percentages:
    
    results[str(percentage)] = {'correlation':[], 'frobenius':[]}
    name = 'Results/Real/Planaria_rdmseed'+str(random_seed)+'_percent'+str(percentage)+'_nmbrpermutation'+str(nmbr_permutation)+'_'
    with open(name+'corr_dict.pkl', 'rb') as f:
        results[str(percentage)]['correlation'] = pickle.load(f)
    with open(name+'frobenius_dict.pkl', 'rb') as f:
        results[str(percentage)]['frobenius'] = pickle.load(f)
    with open(name+'time_dict.pkl', 'rb') as f:
        results[str(percentage)]['time'] = pickle.load(f)   
        
    non_valid_perm = [] # to remove numerical issues and outliers
    values = results[str(percentage)]['frobenius']['steiner']
    for i in range(len(values)):
        if math.isnan(values[i]):
            non_valid_perm.append(i) 
        elif (percentage==80 and results['80']['time']['nj'][i]>170): #remove NJ time outliers
            non_valid_perm.append(i)
        elif (percentage == 92.5 and i==40) or (percentage == 90 and i==26): #remove MST time outliers
            non_valid_perm.append(i)
    valid_perm = np.arange(len(values))
    valid_perm = np.delete(valid_perm, non_valid_perm)  
    results[str(percentage)]['valid_perm'] = valid_perm

In [17]:
# Rearanging data structure
nj_frobenius = {}
steiner_frobenius = {}
mst_frobenius = {}

nj_time = {}
steiner_time = {}
mst_time = {}

for percentage in percentages:

    nj_frobenius[str(percentage)] = np.array(results[str(percentage)]['frobenius']['nj'])[results[str(percentage)]['valid_perm']]
    steiner_frobenius[str(percentage)] = np.array(results[str(percentage)]['frobenius']['steiner'])[results[str(percentage)]['valid_perm']]  
    mst_frobenius[str(percentage)] = np.array(results[str(percentage)]['frobenius']['mst'])[results[str(percentage)]['valid_perm']]

    nj_time[str(percentage)] = np.array(results[str(percentage)]['time']['nj'])[results[str(percentage)]['valid_perm']]
    steiner_time[str(percentage)] = np.array(results[str(percentage)]['time']['steiner'])[results[str(percentage)]['valid_perm']]  
    mst_time[str(percentage)] = np.array(results[str(percentage)]['time']['mst'])[results[str(percentage)]['valid_perm']]  
    
    
    #to take the mean instead of the "official" frobenius, so that we can compare across percentages
    threshold = int((100-percentage)/100 * data.shape[0]) # data has 21452 points    
    nj_frobenius[str(percentage)] = nj_frobenius[str(percentage)] / np.sqrt(threshold)
    steiner_frobenius[str(percentage)] = steiner_frobenius[str(percentage)] / np.sqrt(threshold)
    mst_frobenius[str(percentage)] = mst_frobenius[str(percentage)] / np.sqrt(threshold)

# Plot the results for the cell age prediction task

## Plot the distance error

In [24]:
#Plot performances

sns.set(style="darkgrid")

x = np.linspace(75, 95, 9)

transparency = 0.1
divide_factor = 5.0

xlabels = ['75', '', '80', '', '85', '', '90', '', '95']
fig, ax = plt.subplots()

y_nj = [nj_frobenius[str(percentage)].mean() for percentage in percentages][::-1]
std_nj = np.array([nj_frobenius[str(percentage)].std() for percentage in percentages][::-1])
std_nj = std_nj / divide_factor
ax.fill_between(x, y_nj - std_nj, y_nj + std_nj, color='tab:green', alpha=transparency)
ax.plot(x, y_nj, color='tab:green', label='NJ')

y_steiner = [steiner_frobenius[str(percentage)].mean() for percentage in percentages][::-1]
std_steiner = np.array([steiner_frobenius[str(percentage)].std() for percentage in percentages][::-1])
std_steiner = std_steiner / divide_factor
ax.fill_between(x, y_steiner - std_steiner, y_steiner + std_steiner, color='tab:red', alpha=transparency)
ax.plot(x, y_steiner, color='tab:red', label = 'HyperSteiner')

y_mst = [mst_frobenius[str(percentage)].mean() for percentage in percentages][::-1]
std_mst = np.array([mst_frobenius[str(percentage)].std() for percentage in percentages][::-1])
std_mst = std_mst / divide_factor
ax.fill_between(x, y_mst - std_mst, y_mst + std_mst, color='tab:blue', alpha=transparency)
ax.plot(x, y_mst, color='tab:blue', label='MST')

plt.xlabel('Percentage of removed data')
plt.ylabel('Distance error')
title = 'Results/Real/results_realdata_performances'

plt.xticks(x, xlabels)
ax.legend(loc='upper center').get_frame().set_facecolor('white')

ax.set_yticks([0.17, 0.18, 0.19])
ax.set_xlim(75, 95)
ax.patch.set_alpha(0.5)

plt.savefig(title+'.pdf')
plt.show()

## Plot the CPU time

In [ ]:
# Plot time

x = np.linspace(75, 95, 9)
transparency = 0.1

xlabels = ['75', '', '80', '', '85', '', '90', '', '95']
fig, ax = plt.subplots()

y_nj = [nj_time[str(percentage)].mean() for percentage in percentages][::-1]
std_nj = np.array([nj_time[str(percentage)].std() for percentage in percentages][::-1])
ax.fill_between(x, y_nj - std_nj, y_nj + std_nj, color='tab:green', alpha=transparency)
ax.plot(x, y_nj, color='tab:green', label='NJ')

y_steiner = [steiner_time[str(percentage)].mean() for percentage in percentages][::-1]
std_steiner = np.array([steiner_time[str(percentage)].std() for percentage in percentages][::-1])
ax.fill_between(x, y_steiner - std_steiner, y_steiner + std_steiner, color='tab:red', alpha=transparency)
ax.plot(x, y_steiner, color='tab:red', label = 'HyperSteiner')

y_mst = [mst_time[str(percentage)].mean() for percentage in percentages][::-1]
std_mst = np.array([mst_time[str(percentage)].std() for percentage in percentages][::-1])
ax.fill_between(x, y_mst - std_mst, y_mst + std_mst, color='tab:blue', alpha=transparency)
ax.plot(x, y_mst, color='tab:blue', label = 'MST')

plt.xlabel('Percentage of removed data')
plt.ylabel('CPU time')
title = 'Results/Real/results_realdata_time'

plt.xticks(x, xlabels)
ax.set_yticks([0, 50, 100, 150])
#ax.legend(loc='upper center')
ax.set_xlim(75, 95)
ax.patch.set_alpha(0.5)

plt.savefig(title+'.pdf')
plt.show()